In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
RANDOM_STATE = 1
V = 1


In [ ]:
ROOT = ".."
df = pd.read_csv(f"{ROOT}/data_calculated/02x_m_sin5w.csv")
DESCRIPTOR_NAMES = ['x1'] #,'x2']
TARGET_NAME = "y"
Xraw = df[DESCRIPTOR_NAMES].values
y = df[TARGET_NAME].values


In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(Xraw)
y = y - np.mean(y)


In [ ]:
plt.plot(X[:, 0], y, ".-")


In [ ]:
kernel = RBF(length_scale=1)
reg = GaussianProcessRegressor(kernel=kernel, optimizer=None)


In [ ]:
iopt = np.argmax(y)


In [ ]:
from BO_misc import plot_GPR

random.seed(RANDOM_STATE)
idx = range(X.shape[0])
action = random.sample(idx, 3)
final_action = False
for it in range(20):
    Xtrain = X[action]
    ytrain = y[action]
    reg.fit(Xtrain, ytrain)
    yp_mean, yp_std = reg.predict(X, return_std=True)
    acq = yp_mean + yp_std*np.sqrt(V*it)
    ia = np.argmax(acq)
    action = np.hstack([action, ia])
    plot_GPR(X, y, Xtrain, ytrain, yp_mean, yp_std, acq, it+1, ia)
    if final_action:
        break
    if iopt in action:
        print(f"iteration {it} action={action}")
        final_action = True
